In [1]:
import fiona
import rasterio
import rasterio.mask
import os
from rasterio.windows import Window
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm

In [2]:
src_raster_path = os.getenv('IMAGE_PATH')
shp_file_path = os.getenv('CROPS_SHAPE_PATH')
output_dir = 'negative_crop'
block_size = 1024

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [3]:
def process_block(i, j, out_image, out_transform, out_meta):
    """
    Função para processar e salvar um bloco específico do raster.
    """
    
    block_height = min(block_size, out_image.shape[1] - i)
    block_width = min(block_size, out_image.shape[2] - j)
    
    window = Window(j, i, block_width, block_height)

    block = out_image[:, i:i + block_height, j:j + block_width]

    block_transform = out_transform * window.transform
    block_meta = out_meta.copy()
    block_meta.update({
        "height": block_height,
        "width": block_width,
        "transform": block_transform
    })

    block_output_path = os.path.join(output_dir, f"block_{i}_{j}.tif")

    with rasterio.open(block_output_path, "w", **block_meta) as dest:
        dest.write(block)

    print(f"Bloco salvo: {block_output_path}")

In [4]:
def main():
    with fiona.open(shp_file_path, "r") as shapefile:
        shapes = [feature["geometry"] for feature in shapefile]

    with rasterio.open(src_raster_path) as src:
        out_image, out_transform = rasterio.mask.mask(src, shapes, invert=True)
        out_meta = src.meta
        
        out_meta.update({
            "driver": "GTiff",
            "height": out_image.shape[1],
            "width": out_image.shape[2],
            "transform": out_transform
        })

        height, width = out_image.shape[1], out_image.shape[2]

        tasks = [(i, j) for i in range(0, height, block_size) for j in range(0, width, block_size)]

        with ThreadPoolExecutor(max_workers=2) as executor:
            list(tqdm(executor.map(lambda args: process_block(*args, out_image, out_transform, out_meta), tasks),
                      total=len(tasks),
                      desc="Processando blocos"))

if __name__ == "__main__":
    main()

Processando blocos:   0%|          | 0/210 [00:00<?, ?it/s]


AttributeError: 'Window' object has no attribute 'transform'